# Read in semi-structured data with pandas

When analyzing software systems in a Software Analytics style with [pandas](https://pandas.pydata.org/), you might might face data that isn't yet in a tabular format you can easily read in. In this notebook, I'll show you how you can read in semi-structured data. It's a set of tipps and tricks how you can
* extract d

## Dataset

In our case, we want to analyze data from a version control system. The dataset was generated from the Git repository

`JavaOnAutobahn/spring-petclinic`

with

`git log --stat > git_log_stat.log`.

This command exports the history of the Git repository including some information about the file changes per commit. Here is an excerpt form this created dataset:

```
commit 4d3d9de655faa813781027d8b1baed819c6a56fe
Author: Markus Harrer <feststelltaste@googlemail.com>
Date:   Tue Mar 5 22:32:20 2019 +0100

    add virtual bounded contexts

20	1	jqassistant/business.adoc
```

For each commit, we have this text fragment. The dataset isn't structured data in a tabular way but a more row-based style of data. Each row contains a different kind of information, e.g, the commit id, the author's name, the commit date, the commit message (in the worst case: spread across multiple lines!) as well as the changed files with the number of added and deleted lines of code.

**The question is: Can we get this kind of data into a pandas DataFrame?**

Let's see!

_Note: You can also export data from Git with the `--format` options to create a tabular output. Use this to save you some headaches. But there might be data sources that don't have this option. So it's a good idea to be prepared!_

_Feedback: This notebook shows my brute force approach for handling semi-structured data with pandas. I would be very happy if you have some suggestions on how to improve this in a more simple way!_

## Read in the data

We first load this semi-structured data into a DataFrame. We use a little trick for doing this. Using the `newline` symbol as separator reads that data in line by line.

In [1]:
!cp ../../joa_spring-petclinic/git_log_numstat.log datasets/git_log_raw_stats_spring_petclinic.log

In [2]:
import pandas as pd

log = pd.read_csv(
    "datasets/git_log_raw_stats_spring_petclinic.log",
    sep="\n",
    names=['raw'])
log.head()

,raw
0,commit 4d3d9de655faa813781027d8b1baed819c6a56fe
1,Author: Markus Harrer <feststelltaste@googlema...
2,Date: Tue Mar 5 22:32:20 2019 +0100
3,add virtual bounded contexts
4,20\t1\tjqassistant/business.adoc


## Information Extraction Adventure
Now we have to extract each bit of information accordingly. It's a thankless job. But it works quite well in most cases.

### Extract a row to a separate column

We start with the rows contain information that can be put into separate columns rather easily. For this, we look for markers e.g., at the beginning of a row. We then can use these markers to find the rows we like to extract and apply custom string splittings to them. This works for the information about the commit id, the author's name as well as the commit date.

In [3]:
log['sha'] = log.loc[log['raw'].str.startswith("commit ")]['raw'].str.split("commit ").str[1]
log['author'] = log.loc[log['raw'].str.startswith("Author: ")]['raw'].str.split("Author: ").str[1]
log['timestamp'] = log.loc[log['raw'].str.startswith("Date: ")]['raw'].str.split("Date: ").str[1]
log.head()

,raw,sha,author,timestamp
0,commit 4d3d9de655faa813781027d8b1baed819c6a56fe,4d3d9de655faa813781027d8b1baed819c6a56fe,NaN,NaN
1,Author: Markus Harrer <feststelltaste@googlema...,NaN,Markus Harrer <feststelltaste@googlemail.com>,NaN
2,Date: Tue Mar 5 22:32:20 2019 +0100,NaN,NaN,Tue Mar 5 22:32:20 2019 +0100
3,add virtual bounded contexts,NaN,NaN,NaN
4,20\t1\tjqassistant/business.adoc,NaN,NaN,NaN


### Extract further rows to one column
Next, we want to handle the multiline commit messages. These are also kind of marked by four consecutive whitespacs at the beginning. So we can extract them also with the same approach as above (ugly, but it works!).

In [4]:
log['message'] = log.loc[log['raw'].str.startswith(" "*4)]['raw'].str[4:]
log.head()

,raw,sha,author,timestamp,message
0,commit 4d3d9de655faa813781027d8b1baed819c6a56fe,4d3d9de655faa813781027d8b1baed819c6a56fe,NaN,NaN,NaN
1,Author: Markus Harrer <feststelltaste@googlema...,NaN,Markus Harrer <feststelltaste@googlemail.com>,NaN,NaN
2,Date: Tue Mar 5 22:32:20 2019 +0100,NaN,NaN,Tue Mar 5 22:32:20 2019 +0100,NaN
3,add virtual bounded contexts,NaN,NaN,NaN,add virtual bounded contexts
4,20\t1\tjqassistant/business.adoc,NaN,NaN,NaN,NaN


Note: We still have to treat commit messages that span across multiple rows. We have to care about that later on.

### Extract multiple columns from multiple row
Now for the remaining rows: The information about the additions and deletions per filename. This is a little bit tricky in three ways:

* There is no dedicated marker for the file statistics
* There are multiple information about the modified file in one row (added & deleted lines as well as the filename)
* There are multiple rows for all the changed files within one commit

We can handle this step by step. First, we mark the rows that haven't been extracted yet into separate columns by creating a new column `no_entry` with `True` entries for those.

In [5]:
log['no_entry'] = \
    log['sha'].isna() & \
    log['author'].isna() & \
    log['timestamp'].isna() & \
    log['message'].isna()
log.head()

,raw,sha,author,timestamp,message,no_entry
0,commit 4d3d9de655faa813781027d8b1baed819c6a56fe,4d3d9de655faa813781027d8b1baed819c6a56fe,NaN,NaN,NaN,False
1,Author: Markus Harrer <feststelltaste@googlema...,NaN,Markus Harrer <feststelltaste@googlemail.com>,NaN,NaN,False
2,Date: Tue Mar 5 22:32:20 2019 +0100,NaN,NaN,Tue Mar 5 22:32:20 2019 +0100,NaN,False
3,add virtual bounded contexts,NaN,NaN,NaN,add virtual bounded contexts,False
4,20\t1\tjqassistant/business.adoc,NaN,NaN,NaN,NaN,True


In the next step, we need to signal which file statistics information belongs to which commit. Luckily, there is a marker from this that we've already extracted: the `sha` column. This information is also the start of a commit entry. So we can use this entry to mark all the follow up entries of a commit to signal that these rows belong together.

In [6]:
log['sha'] = log['sha'].fillna(method="ffill")
log.head()

,raw,sha,author,timestamp,message,no_entry
0,commit 4d3d9de655faa813781027d8b1baed819c6a56fe,4d3d9de655faa813781027d8b1baed819c6a56fe,NaN,NaN,NaN,False
1,Author: Markus Harrer <feststelltaste@googlema...,4d3d9de655faa813781027d8b1baed819c6a56fe,Markus Harrer <feststelltaste@googlemail.com>,NaN,NaN,False
2,Date: Tue Mar 5 22:32:20 2019 +0100,4d3d9de655faa813781027d8b1baed819c6a56fe,NaN,Tue Mar 5 22:32:20 2019 +0100,NaN,False
3,add virtual bounded contexts,4d3d9de655faa813781027d8b1baed819c6a56fe,NaN,NaN,add virtual bounded contexts,False
4,20\t1\tjqassistant/business.adoc,4d3d9de655faa813781027d8b1baed819c6a56fe,NaN,NaN,NaN,True


OK, we see, this seems to get somehow complicated. So let's create a separate DataFrame for this called `sha_files`, were we just treat the file statistics. This DataFrame contains now for each commit all the change information for each changed file.

In [7]:
sha_files = log[log['no_entry']][['sha', 'raw']]
sha_files = sha_files.set_index('sha')
sha_files.head()

,raw
sha,
4d3d9de655faa813781027d8b1baed819c6a56fe,20\t1\tjqassistant/business.adoc
4d3d9de655faa813781027d8b1baed819c6a56fe,1\t1\tsrc/main/java/org/springframework/sample...
4d3d9de655faa813781027d8b1baed819c6a56fe,2\t0\tsrc/main/java/org/springframework/sample...
4d3d9de655faa813781027d8b1baed819c6a56fe,2\t1\tsrc/main/java/org/springframework/sample...
4d3d9de655faa813781027d8b1baed819c6a56fe,2\t0\tsrc/main/java/org/springframework/sample...


We are now able to focus on the files statistics. We can split the `raw` entries with the tabular symbol and throw away the raw data. This fives as us nicely formatted DataFrame with the files statistics' information.

In [8]:
sha_files[['additions', 'deletions', 'filename']] = sha_files['raw'].str.split("\t", expand=True)
del(sha_files['raw'])
sha_files.head()

,additions,deletions,filename
sha,,,
4d3d9de655faa813781027d8b1baed819c6a56fe,20,1,jqassistant/business.adoc
4d3d9de655faa813781027d8b1baed819c6a56fe,1,1,src/main/java/org/springframework/samples/petc...
4d3d9de655faa813781027d8b1baed819c6a56fe,2,0,src/main/java/org/springframework/samples/petc...
4d3d9de655faa813781027d8b1baed819c6a56fe,2,1,src/main/java/org/springframework/samples/petc...
4d3d9de655faa813781027d8b1baed819c6a56fe,2,0,src/main/java/org/springframework/samples/petc...


Next, we want to join this data with the other, bigger `log` DataFrame that contains all the other information about the commits. This means we have to arrange the other DataFrame so that we can join our newly created `sha_files` DataFrame. We can accomplish this by groupby by the `sha` columns. We also try to reduce complexity by just preserving the meta information with the author and the timestamp for now.

In [9]:
meta_data = log.groupby('sha')[['author', 'timestamp']].first()
meta_data.head()

,author,timestamp
sha,,
024811d252f8d8218e6795d46203cff25971bc19,Mic <misvy@vmware.com>,Thu Mar 14 18:04:36 2013 +0800
0365d34d2977dd24ec0bb3e8b0edff5694908c80,Markus Harrer <feststelltaste@googlemail.com>,Mon Nov 12 10:28:34 2018 +0100
0504ec9fe345d9d34b15c374333f709fb147e6d6,thinksh <thinkshihang@gmail.com>,Wed Feb 3 23:19:46 2016 -0500
053c84ecc95b246ef4a40fb3d4304e8908604af4,Mic <misvy@vmware.com>,Mon Feb 3 09:31:44 2014 +0800
057015c14cce4791ff309419de8a8bd6339fd6e7,Mic <misvy@vmware.com>,Fri Feb 15 15:31:04 2013 +0800


With both DataFrames having the same index column `sha`, we can now join DataFrames. We set the join method to `right` because we have multiple file statistics entries for each commit. This expands the `meta_data` DataFrame, i.e., duplicates each meta data entry for a file statistics entry.

In [10]:
changes = meta_data.join(sha_files, how='right')
changes.head()

,author,timestamp,additions,deletions,filename
sha,,,,,
024811d252f8d8218e6795d46203cff25971bc19,Mic <misvy@vmware.com>,Thu Mar 14 18:04:36 2013 +0800,1,1,src/main/java/org/springframework/samples/petc...
0365d34d2977dd24ec0bb3e8b0edff5694908c80,Markus Harrer <feststelltaste@googlemail.com>,Mon Nov 12 10:28:34 2018 +0100,1,1,pom.xml
0504ec9fe345d9d34b15c374333f709fb147e6d6,thinksh <thinkshihang@gmail.com>,Wed Feb 3 23:19:46 2016 -0500,1,1,src/main/resources/db/mysql/petclinic_db_setup...
053c84ecc95b246ef4a40fb3d4304e8908604af4,Mic <misvy@vmware.com>,Mon Feb 3 09:31:44 2014 +0800,1,1,pom.xml
057015c14cce4791ff309419de8a8bd6339fd6e7,Mic <misvy@vmware.com>,Fri Feb 15 15:31:04 2013 +0800,1,18,.springBeans


Alright, we are almost done. Hang in there!

### Combine multiple rows to one entry in a column

We still have to treat the commit messages that span across multiple lines. So back to the message information. Thanks to the `sha` column, we are able to concatenate all the messages that belong to one commit as well by joining the message's party in one single row.

In [11]:
sha_msg = log.dropna(subset=['message']).groupby('sha')['message'].apply(' '.join)
sha_msg.head()

sha
024811d252f8d8218e6795d46203cff25971bc19                        simplifying access to Integer
0365d34d2977dd24ec0bb3e8b0edff5694908c80             downgrade jqassistant due to weird error
0504ec9fe345d9d34b15c374333f709fb147e6d6    Update petclinic_db_setup_mysql.txt Correct in...
053c84ecc95b246ef4a40fb3d4304e8908604af4                             migrated to Spring 4.0.1
057015c14cce4791ff309419de8a8bd6339fd6e7    Spring MVC Test Framework and migration to Spr...
Name: message, dtype: object

### Combining commit messages and change information

Finally, we also join this separate Series with the main DataFrame. Done!

In [12]:
changes = changes.join(sha_msg)
changes.head()

,author,timestamp,additions,deletions,filename,message
sha,,,,,,
024811d252f8d8218e6795d46203cff25971bc19,Mic <misvy@vmware.com>,Thu Mar 14 18:04:36 2013 +0800,1,1,src/main/java/org/springframework/samples/petc...,simplifying access to Integer
0365d34d2977dd24ec0bb3e8b0edff5694908c80,Markus Harrer <feststelltaste@googlemail.com>,Mon Nov 12 10:28:34 2018 +0100,1,1,pom.xml,downgrade jqassistant due to weird error
0504ec9fe345d9d34b15c374333f709fb147e6d6,thinksh <thinkshihang@gmail.com>,Wed Feb 3 23:19:46 2016 -0500,1,1,src/main/resources/db/mysql/petclinic_db_setup...,Update petclinic_db_setup_mysql.txt Correct in...
053c84ecc95b246ef4a40fb3d4304e8908604af4,Mic <misvy@vmware.com>,Mon Feb 3 09:31:44 2014 +0800,1,1,pom.xml,migrated to Spring 4.0.1
057015c14cce4791ff309419de8a8bd6339fd6e7,Mic <misvy@vmware.com>,Fri Feb 15 15:31:04 2013 +0800,1,18,.springBeans,Spring MVC Test Framework and migration to Spr...


## Summary
In this notebook, I showed you how you can also work with non-tabular or semi-structured data with pandas. There are some edge cases in software analysis where you need this. I hope you saw that you can treat this but that this is also complicated. If you face semi-structured data I encourage you to ask for an alternative representation of data in a tabular style format to avoid this kind of complicated work!